In [87]:
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import json
import datetime
import re

In [3]:
sheet_id = ''
sheet_name = ''
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}' 
data_from_excel = pd.read_csv(url)

In [4]:
data_from_excel.reset_index(inplace=True)

In [6]:
data = data_from_excel

In [9]:
data.dropna(subset=["Link Video"], inplace=True)
Tiktok_data = data[data['Link Video'].str.contains("tiktok.com")]

# Tiktok

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.tiktok.com/login/")

In [ ]:
begin_time = datetime.datetime.now()

viewcount_list = []

for i in Tiktok_data["Link Video"].values:
    url = ('view-source:' + i)
    driver.get(url)
    try:
        data_j = driver.find_element_by_tag_name('body').text
        soup = BeautifulSoup(data_j)

        info = soup.find("script", type="application/json").contents[0]
        json_data = json.loads(info)

        view = json_data["props"]["pageProps"]["itemInfo"]["itemStruct"]["stats"]["playCount"]

        item = {    
                    "Link Video": i,
                    "Số View": view,
        }

        viewcount_list.append(item)
    except:
        print(i)

df = pd.DataFrame(viewcount_list)
df = Tiktok_data.merge(df, on="Link Video", how="left")  
Tiktok_time = begin_time
df.to_pickle("./tiktok.pkl")

print(datetime.datetime.now() - begin_time)

# Instagram

In [22]:
insta_data = data[data['Link Video'].str.contains('instagram')]

# get rid of ?utm_...
insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]

<ipython-input-22-3b6a3fb73241>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  insta_data['Link Video'] = insta_data['Link Video'].str.split("?", expand=True)[0]


In [23]:
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/accounts/login/")

In [24]:
insta_list = []

for link in insta_data["Link Video"].values:
    base_url = link
    url_json = (base_url + '?__a=1')
    driver.get(url_json)
    
    data_j = driver.find_element_by_tag_name('body').text
    try:
        data_json = json.loads(data_j)

        view = data_json["graphql"]['shortcode_media']['video_view_count']

        item = {
                    "Link Video": link,
                    "Số View": view,
        }

        insta_list.append(item)
        time.sleep(.5)
    except:
        print(link + " Cannot get insta info!")
    
df = pd.DataFrame(insta_list)
df = insta_data.merge(df, on="Link Video", how="left")
df.to_pickle("insta.pkl")
insta_datetime = datetime.datetime.now()

# Facebook

In [25]:
fb_data = data[data['Link Video'].str.startswith('https://www.facebook.com/')]
fb_data = fb_data[fb_data['Link Video'].str.contains("/videos/")]

In [89]:
driver = webdriver.Chrome()
driver.get("https://www.facebook.com/")

In [90]:
fb_list = []

for link in fb_data["Link Video"].values:
 
    driver.get('view-source:' + link)
    
    try:
        data_j = driver.find_element_by_tag_name('body').text

        view = int(re.search('"video_view_count":\d+', data_j).group()[19:])
        
        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        fb_list.append(item)
    
    except:
        print(link)
        
df = pd.DataFrame(fb_list)
df = fb_data.merge(df, on="Link Video", how="left")
df.to_pickle("fb.pkl")

fb_datetime = datetime.datetime.now()

# Youtube

In [102]:
youtube_data = data[data['Link Video'].str.startswith('https://www.youtube.com/')]

In [113]:
driver = webdriver.Chrome()
youtube_list = []

for link in youtube_data["Link Video"].values:
    
    driver.get('view-source:' + link)

    try:
        data_j = driver.find_element_by_tag_name('body').text
        view = int(re.search('"viewCount":"\d+"', data_j).group()[13:-1])

        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        youtube_list.append(item)
    except:
        print(link)
        
df = pd.DataFrame(youtube_list)
df = youtube_data.merge(df, on="Link Video", how="left")
df.to_pickle("youtube.pkl")

youtube_datetime = datetime.datetime.now()

In [115]:
tiktok = pd.read_pickle("tiktok.pkl")
insta = pd.read_pickle("insta.pkl")
fb = pd.read_pickle("fb.pkl")
youtube = pd.read_pickle("youtube.pkl")

In [118]:
tiktok_insta_fb_yt = pd.concat([tiktok, insta, fb, youtube])
tiktok_insta_fb_yt["Số View"] = tiktok_insta_fb_yt["Số View"] / 1000

In [119]:
tiktok_insta_fb_yt = tiktok_insta_fb_yt[["index", "Số View"]]

In [127]:
export_df = data_from_excel.merge(tiktok_insta_fb_yt, on="index", how="left")

In [128]:
time_df = pd.DataFrame({"Ngày giờ" :["Tiktok : " + str(Tiktok_time), "Insta : " + str(insta_datetime), "Fb : " + str(fb_datetime), "Youtube : " + str(youtube_datetime)]})
export_df = pd.concat([export_df, time_df], axis=1)
export_df.rename(columns={"Số View": "Số view (đơn vị 1000)"}, inplace=True)
push_df = export_df[[""]]

In [129]:
final_push = push_df[~push_df.duplicated()]

In [130]:
import gspread
from gspread_dataframe import set_with_dataframe

# ACCES GOOGLE SHEET
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('Python') #->gg sheet name 

# APPEND DATA TO SHEET
your_dataframe = final_push
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET